# Script spacy informe Rettig

##### 1. import tabla eventos del sparcing

In [2]:
import pandas as pd

# correr el codigo de sparcing
%run script_parsing.ipynb


events = pd.read_csv('data_out/events_rettig.csv')
events = events['events'].tolist()

html object created
HTML content has been exported to data_out/output.html.


#### 2. Maqueta spaCy summary y NER

Proceso la base de datos iterando por evento con un modelo de lenguaje de spacy que tiene tokenizer y NER. Con el tokenizer hago un summary y con el NER le hago detectar todas las NER. 

La base de datos no está bien lista, hay que mejorar el modelo del summary y hay que destilar las entidades, pero muestra el tipo de output que se puede hacer. 

In [3]:
import spacy
from spacy.lang.es.stop_words import STOP_WORDS
from collections import Counter

# modelo español con NER, tokennizer, etc. 
nlp = spacy.load("es_core_news_sm")

#####
# def funcion para summary y NER 
def generate_summary(text, num_sentences=3):
    # procesar texto 
    doc = nlp(text)
    
    # filtrar stop words de la bagg of words 
    words = [token.text for token in doc if not token.is_stop and not token.is_punct]
    
    # word frequency 
    word_freq = Counter(words)
    
    # sentence frequency 
    sentence_scores = {}
    for sent in doc.sents:
        for word in sent:
            if word.text in word_freq:
                if sent not in sentence_scores:
                    sentence_scores[sent] = word_freq[word.text]
                else:
                    sentence_scores[sent] += word_freq[word.text]
    
    

    # top sentences 
    top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    
    # pegamos el summary
    summary = ' '.join(str(sent) for sent in top_sentences)

    ### NER
    # extraer entidades
    entities = [ent for ent in doc.ents if ent.text.lower() not in STOP_WORDS]
    
    # pegar tag con entidad
    entity_pairs = [(ent.label_, ent.text) for ent in entities]
    ###

    # output 
    return summary, entity_pairs
#####

# iteramos la función sobre cada evento
eventos_summary_ner = []
for event in events:
    summary, entity_pairs = generate_summary(event)
    eventos_summary_ner.append((event, summary, entity_pairs))

/Users/juanipiquer/anaconda3/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'es_core_news_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [4]:
# export excel
eventos_summary_ner = pd.DataFrame(eventos_summary_ner)

eventos_summary_ner.to_excel('data_out/eventos_summary_ner.xlsx')